In [1]:
import shutil
from pathlib import Path
from PIL import Image
from tqdm.notebook import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
dir1 = Path("100scans")
dir2 = Path("300scans")
output_dir = Path("Scans")
output_dir.mkdir(parents=True, exist_ok=True)

def resize_and_save_image(source, destination):
    with Image.open(source) as img:
        img.thumbnail((1024, 1024), Image.LANCZOS)
        img.save(destination)

unique_folders_in_output = set()
dir1_childs = {f.name for f in dir1.iterdir() if f.is_dir()}
same_folder_name_count = len(dir1_childs & {f.name for f in dir2.iterdir() if f.is_dir()})

total_files_dir1 = sum(1 for _ in dir1.rglob("*.png"))
total_files_dir2 = sum(1 for _ in dir2.rglob("*.png") if _.parent.relative_to(dir2).parts[0] not in dir1_childs)
total_files = total_files_dir1 + total_files_dir2

In [3]:
with tqdm(total=total_files, unit="file") as pbar:
    with ThreadPoolExecutor(max_workers=4) as executor:
        futures = []
        for source in dir1.rglob("*.png"):
            destination = output_dir / source.relative_to(dir1)
            destination.parent.mkdir(parents=True, exist_ok=True)
            unique_folders_in_output.add(destination.parent.relative_to(output_dir).parts[0])
            futures.append(executor.submit(shutil.copy2, source, destination))
        for source in dir2.rglob("*.png"):
            first_level_folder = source.relative_to(dir2).parts[0]
            if first_level_folder in dir1_childs:
                continue
            destination = output_dir / source.relative_to(dir2)
            destination.parent.mkdir(parents=True, exist_ok=True)
            futures.append(executor.submit(resize_and_save_image, source, destination))
            unique_folders_in_output.add(first_level_folder)
        for future in as_completed(futures):
            pbar.update()

print(f"Unique child folders created in '{output_dir}': {len(unique_folders_in_output)}")
print(f"Same named child folders in '{dir1}' and '{dir2}': {same_folder_name_count}")

  0%|          | 0/9922 [00:00<?, ?file/s]

Unique child folders created in 'Scans': 369
Same named child folders in '100scans' and '300scans': 19
